## Working with pandas dataframes

I am trying to extract the name information using the columns (as a whole, if that makes sense).  I have a function works if I iterate through the rows and treat each 'Name' individually.

In [ ]:
import numpy as np
import pandas as pd
import sqlite3
import os
import feather

In [ ]:
apsim_sourcedir = "/OSM/CBR/AG_WHEATTEMP/source"
apsim_outfiledir = "/OSM/CBR/AG_WHEATTEMP/work"
metfile_sourcedir = "/OSM/CBR/AG_WHEATTEMP/work/ApsimNG-test/APSIM_run/met"

In [ ]:
dbfile_df = pd.DataFrame(columns=['filename'])
dbfile_df.filename = sorted(apsim_sourcedir+'/'+f for f in os.listdir(apsim_sourcedir) if f.endswith('.db'))
print(dbfile_df.head())

In [ ]:
#we are going to work with just the first filename
print(dbfile_df.filename[0])
con = sqlite3.connect(dbfile_df.filename[0])
cur = con.cursor()

In [ ]:
strSql = "SELECT ID as SimulationID, Name FROM _Simulations"
dfSim = pd.read_sql_query(strSql, con, index_col = 'SimulationID')
print(dfSim.shape)
dfSim

In [ ]:
#An alternative method, using grep is done below
#dfSim2 = dfSim.join(pd.DataFrame(dfSim.Name.str.rsplit('_', expand=True, n=1)))
#dfSim2.rename(columns={0: 'newName', 1: 'sowDate'}, inplace=True)

#dfSim2 = dfSim2.join(pd.DataFrame(dfSim2.newName.str.split('_', expand=True, n=2)))
#dfSim2.rename(columns={0: 'long', 1: 'lat', 2: 'variety'}, inplace=True)

#dfSim2.drop('newName', axis=1, inplace=True)
#dfSim2

In [ ]:
#pd.options.display.float_format = '{:,.2f}'.format
#dfSim2['long'] = dfSim2['long'].astype(float) / 100
#dfSim2['lat'] = dfSim2['lat'].astype(float) / 100
#dfSim2

While the above works nicely, it may not be the best way to achieve this, might need to do some testing when dealing with more data.  So I will use regular expressions instead.

In [ ]:
#This is done in a single line below
#dfSim['long'] = dfSim['Name'].str.extract("^(\d+)_-?\d+_\S+_\d+-\S+$", expand=True)
#dfSim['lat'] = dfSim['Name'].str.extract("^\d+_(-?\d+)_\S+_\d+-\S+$", expand=True)
#dfSim['variety'] = dfSim['Name'].str.extract("^\d+_-?\d+_(\S+)_\d+-\S+$", expand=True)
#dfSim['sowdate'] = dfSim['Name'].str.extract("^\d+_-?\d+_\S+_(\d+-\S+)$", expand=True)
#dfSim

In [ ]:
#a more optimal way to achieve the above
dfSim[['long','lat','variety','sowdate']] = dfSim['Name'].str.extract("^(?P<lo>\d+)_(?P<la>-?\d+)_(?P<var>\S+)_(?P<da>\d+-\S+)$", expand=True)
dfSim

In [ ]:
#now format the longitude and latitude columns correctly
pd.options.display.float_format = '{:,.2f}'.format
dfSim['long'] = dfSim['long'].astype(float) / 100
dfSim['lat'] = dfSim['lat'].astype(float) / 100



In [ ]:
print(dfSim.shape)
dfSim

#### Now lets filter this by the Variety we are after

In [ ]:
dfSimVar = dfSim[(dfSim.variety=='janz')]
print(dfSimVar.shape)
dfSimVar

#### get a list of Simulation ID's

In [ ]:
simIds = dfSimVar.index.tolist()
simIds

In [ ]:
simIdStr = ', '.join(str(e) for e in simIds)
simIdStr

In [ ]:
dfSimVar['SimID'] = dfSimVar.index
dfSimVar

### Now retrieve the Report Data

We can use the simIds that were collected above

In [ ]:
#This should return all of the columns
strSql = "SELECT * FROM Report \
          WHERE SimulationID IN (" + simIdStr + ") \
          ORDER BY SimulationID"
strSql


In [ ]:
#c = con.cursor()
#c.execute(strSql, simIds)
#varietyRows = c.fetchall()

In [ ]:
#varietyRows = pd.DataFrame(varietyRows)
#varietyRows
#print(varietyRows.shape)

In [ ]:
dfReport = pd.read_sql_query(strSql, con, \
                       index_col="SimulationID"
                      )

In [ ]:
#This should return all of the columns
#strSql = "SELECT SimulationID, [Clock.Today], [Wheat.Leaf.LAI], [Wheat.AboveGround.Wt], \
#          [Wheat.Grain.Wt], [Wheat.Phenology.Zadok.Stage], [Wheat.WaterSupplyDemandRatio], \
#          [Wheat.Root.NUptake], [Wheat.Leaf.Fn] \
#          FROM Report \
#          ORDER BY SimulationID, [Clock.Today]"
#dfReport = pd.read_sql_query(strSql, con, \
#                       index_col="SimulationID"
#                      )


In [ ]:
print(dfReport.shape)
dfReport

In [ ]:
#### Need to re-format the Clock.Today column so that it only includes the date
dfReport['date'] = pd.to_datetime(dfReport['Clock.Today']).apply(lambda x:x.strftime('%d/%m/%Y'))
dfReport['SimID'] = dfReport.index
dfReport

In [ ]:
#strSql = "SELECT DISTINCT SimulationID, [Clock.Today], [Wheat.Leaf.LAI], [Wheat.AboveGround.Wt], \
#          [Wheat.Grain.Wt], [Wheat.Phenology.Zadok.Stage], [Wheat.WaterSupplyDemandRatio], \
#          [Wheat.Root.NUptake], [Wheat.Leaf.Fn] \
#          FROM Report \
#          WHERE SimulationID = 1;"
#dfReport2 = pd.read_sql_query(strSql, con, \
#                       parse_dates = {"Clock.Today": '%Y-%m-%D %T'}, \
#                       index_col="Clock.Today"
#                      )

In [ ]:
#print(dfReport2.shape)

In [ ]:
#print(dfCombined.shape)
#print(dfReport.shape)

### Now output the results to new ".feather" file

In [ ]:
filename = os.path.splitext(os.path.basename(dbfile_df.filename[0]))[0]
filename = filename.replace(".", "")
#dilename = os.path.splitext(filename)[0]
#filename = apsim_outfiledir + "/" +  filename
outpath = apsim_outfiledir + "/" +  filename + ".feather"
print(outpath)

In [ ]:
feather.write_dataframe(dfCombined, outpath)

In [ ]:
outpathSim = apsim_outfiledir + "/" +  filename  + "_simulations.feather"
print(outpathSim)
outpathReport = apsim_outfiledir + "/" +  filename  + "_report.feather"
print(outpathReport)

In [ ]:
dfSim['SimID'] = dfSim.index
dfSim
#feather.write_dataframe(dfSim, outpath, index=True)


In [ ]:
cols = dfSim.columns.tolist() 
print(cols)
cols = cols[-1:] + cols[1:5] + cols[0:1]
print(cols)
dfSim = dfSim[cols]
dfSim

In [ ]:
feather.write_dataframe(dfSim, outpathSim)
df = feather.read_dataframe(outpathSim)
print(df.shape)
df

In [ ]:
feather.write_dataframe(dfReport, outpathReport)

In [ ]:
df = feather.read_dataframe(outpathReport)
print(df.shape)
df